In [7]:
from dotenv import load_dotenv
import netCDF4 as nc
import numpy as np
import os
import requests
import urllib.request
from urllib.error import HTTPError
import xarray as xr

In [2]:
# Carga las variables de entorno desde el archivo .env
load_dotenv()

# Accede a la contraseña desde las variables de entorno
password = os.getenv('COPERNICUS_PASSWORD')
username = os.getenv('COPERNICUS_USERNAME')

In [32]:

def download_nc_file_with_auth(url, output_file, username, password):
    # Create a password manager with the given username and password
    password_mgr = urllib.request.HTTPPasswordMgrWithDefaultRealm()
    password_mgr.add_password(None, url, username, password)
    handler = urllib.request.HTTPBasicAuthHandler(password_mgr)
    opener = urllib.request.build_opener(handler)
    
    try:
        # Open the URL with the configured opener
        response = opener.open(url)
        
        with open(output_file, 'wb') as f:
            f.write(response.read())
        
        print(f"Downloaded {url} to {output_file}")
    except HTTPError as e:
        print(f"Failed to download {url}. Status code: {e.code}, Reason: {e.reason}")

def slice_ds(lat_min, lat_max, lon_min, lon_max, ds):

    lat_min_ds = ds.lat.values.min()
    lat_max_ds = ds.lat.values.max()
    lat_len_ds = len(ds.lat.values)

    lon_min_ds = ds.lon.values.min()
    lon_max_ds = ds.lon.values.max()
    lon_len_ds = len(ds.lon.values)

    lat_coord = np.linspace(lat_max_ds, lat_min_ds, lat_len_ds)
    lon_coord = np.linspace(lon_min_ds, lon_max_ds, lon_len_ds)

    new_lat_min = np.absolute(lat_coord-lat_min).argmin()
    new_lat_max = np.absolute(lat_coord-lat_max).argmin()
    new_lon_min = np.absolute(lon_coord-lon_min).argmin()
    new_lon_max = np.absolute(lon_coord-lon_max).argmin()

    return ds.NDVI[0, new_lat_max:new_lat_min, new_lon_min:new_lon_max]

In [33]:
link_1 = f"https://land.copernicus.vgt.vito.be/manifest/ndvi_v3_1km/manifest_cgls_ndvi_v3_1km_latest.txt" #2020 - 1999
link_2 = f"https://land.copernicus.vgt.vito.be/manifest/ssm_v1_1km/manifest_cgls_ssm_v1_1km_latest.txt" #SSM 2014
link_3 = f"https://land.copernicus.vgt.vito.be/manifest/fapar_v2_1km/manifest_cgls_fapar_v2_1km_latest.txt" #FAPAR 1999 - 202006

link_4 = f"https://land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Indicators/NDVI_1km_V2_LTS/1999/12/21/NDVI-LTS_1999-2017-1221_GLOBE_VGT-PROBAV_V2.2.1/c_gls_NDVI-LTS_1999-2017-1221_GLOBE_VGT-PROBAV_V2.2.1.nc"

links = [link_1, link_2, link_3]
folders = ["nvdi", "ssm", "fapar"]

link_dict = {link:folder for link, folder in zip(links, folders)}

main_data_folder = f"../data"

In [35]:


for link, folder in link_dict.items():

    data_folder = f"{main_data_folder}/{folder}"
    if os.path.exists(data_folder) is False:
        os.mkdir(data_folder)

    response = requests.get(link)

    response.raise_for_status()

    nc_files = response.text.split("\n")

    nc_file = nc_files[0]

    #for nc_file in nc_files:

    filename = nc_file.split("/")[-1]

    if ".nc" in filename:
        output_filename = f'{data_folder}/{filename}'
        download_nc_file_with_auth(nc_file, output_filename, username, password)

        ds = xr.open_dataset(output_filename)

        ds_slice = slice_ds(lat_min=36, lat_max=46, lon_min=-10, lon_max=3, ds=ds)


        ds_slice.to_netcdf(output_filename)

        ds.close()
        ds_slice.to_netcdf(f"{data_folder}/sample.nc")

    break





Downloaded https://land.copernicus.vgt.vito.be/PDF/datapool/Vegetation/Indicators/NDVI_1km_V3/2020/06/21/NDVI_202006210000_GLOBE_PROBAV_V3.0.1/c_gls_NDVI_202006210000_GLOBE_PROBAV_V3.0.1.nc to ../data/nvdi/c_gls_NDVI_202006210000_GLOBE_PROBAV_V3.0.1.nc


PermissionError: [Errno 13] Permission denied: '/home/carlossi/zrive/zrive-ds-4q23-olive/data/nvdi/c_gls_NDVI_202006210000_GLOBE_PROBAV_V3.0.1.nc'